##### データセットの場所やバッチサイズなどの定数値の設定

In [2]:
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'


# 使用するデバイス
# GPU を使用しない環境（CPU環境）で実行する場合は DEVICE = 'cpu' とする
DEVICE = 'cuda:0'

# 全ての訓練データを一回ずつ使用することを「1エポック」として，何エポック分学習するか
# 再開モードの場合も, このエポック数の分だけ追加学習される（N_EPOCHSは最終エポック番号ではない）
N_EPOCHS = 20

# 学習時のバッチサイズ
BATCH_SIZE = 100

# 訓練データセット（画像ファイルリスト）のファイル名
TRAIN_DATASET_CSV = './FakeFaces/train_list.csv'

# テストデータセット（画像ファイルリスト）のファイル名
TEST_DATASET_CSV = './FakeFaces/test_list.csv'

# 画像ファイルの先頭に付加する文字列（データセットが存在するディレクトリのパス）
DATA_DIR = './FakeFaces/'

# 画像サイズ
H = 256 # 縦幅
W = 256 # 横幅
C = 3 # 入力画像のチャンネル数（カラー画像なら3，グレースケール画像なら1．なお，正解のマスク画像のチャンネル数は常に1）

# 学習結果の保存先フォルダ
MODEL_DIR = './FakeFaces_models/'

# 学習結果のニューラルネットワークの保存先
MODEL_FILE = os.path.join(MODEL_DIR, 'fakeface_detector_model.pth')

# 中断／再開の際に用いる一時ファイル
CHECKPOINT_EPOCH = os.path.join(MODEL_DIR, 'checkpoint_epoch.pkl')
CHECKPOINT_MODEL = os.path.join(MODEL_DIR, 'checkpoint_model.pth')
CHECKPOINT_OPT = os.path.join(MODEL_DIR, 'checkpoint_opt.pth')

##### ニューラルネットワークモデルの定義

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F


# 畳込み，バッチ正規化，ReLUをセットで行うクラス
class myConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
        super(myConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding)
        self.bn = nn.BatchNorm2d(num_features=out_channels)
    def forward(self, x):
        return F.relu(self.bn(self.conv(x)))


# DeepFake Detection Challenge データセットの顔画像に対しReal/Fake判定を行うニューラルネットワーク
class FakeFaceDetector(nn.Module):

    # C: 入力画像のチャンネル数（1または3と仮定）
    # H: 入力画像の縦幅（32の倍数と仮定）
    # W: 入力画像の横幅（32の倍数と仮定）
    def __init__(self, C, H, W):
        super(FakeFaceDetector, self).__init__()

        # 畳込み層1〜5
        # カーネルサイズ3，ストライド幅1，パディング1の設定なので，これを通しても特徴マップの縦幅・横幅は変化しない
        self.conv1 = myConv2d(in_channels=C, out_channels=8, kernel_size=3, stride=1, padding=1)
        self.conv2 = myConv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv3 = myConv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv4 = myConv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv5 = myConv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)

        # プーリング層1～5
        # 1回適用するごとに特徴マップの縦幅・横幅がそれぞれ 1/2 になる
        # 全部で5回適用することになるので，最終的には都合 1/32 になる -> ゆえに，入力画像の縦幅と横幅を各々32の倍数と仮定している
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.pool3 = nn.MaxPool2d(kernel_size=2)
        self.pool4 = nn.MaxPool2d(kernel_size=2)
        self.pool5 = nn.MaxPool2d(kernel_size=2)

        # 平坦化
        self.flat = nn.Flatten()

        # 全結合層1
        # 畳込み層1～5を通すことにより特徴マップの縦幅・横幅は都合 1/32 になっている．
        # 従って，入力側のパーセプトロン数は 64*(H/32)*(W/32) = H*W/16
        self.fc1 = nn.Linear(in_features=H*W//16, out_features=256)

        # 全結合層2
        self.fc2 = nn.Linear(in_features=256, out_features=2) # 最後はRealかFakeかの2クラス

    def forward(self, x):
        h = self.pool1(self.conv1(x))
        h = self.pool2(self.conv2(h))
        h = self.pool3(self.conv3(h))
        h = self.pool4(self.conv4(h))
        h = self.pool5(self.conv5(h))
        h = self.flat(h)
        h = F.relu(self.fc1(h))
        y = self.fc2(h)
        return y

##### 訓練データセットの読み込み

In [ ]:
import pickle
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
from mylib.data_io import CSVBasedDataset
from mylib.utility import save_datasets, load_datasets_from_file


# 前回の試行の続きを行いたい場合は True にする -> 再開モードになる
RESTART_MODE = False


# 再開モードの場合は，前回使用したデータセットをロードして使用する
if RESTART_MODE:
    train_dataset, valid_dataset = load_datasets_from_file(MODEL_DIR)
    if train_dataset is None:
        print('error: there is no checkpoint previously saved.')
        exit()
    train_size = len(train_dataset)
    valid_size = len(valid_dataset)
    with open(os.path.join(MODEL_DIR, 'fdicts.pkl'), 'rb') as fdicts_file:
        fdicts = pickle.load(fdicts_file)
    n_classes = len(fdicts[1])

# そうでない場合は，データセットを読み込む
else:

    # CSVファイルを読み込み, 訓練データセットを用意
    dataset = CSVBasedDataset(
        filename = TRAIN_DATASET_CSV,
        items = [
            'File Path', # X
            'Label', # Y
        ],
        dtypes = [
            'image', # Xの型
            'label', # Yの型
        ],
        dirname = DATA_DIR,
    )
    with open(os.path.join(MODEL_DIR, 'fdicts.pkl'), 'wb') as fdicts_file:
        pickle.dump(dataset.forward_dicts, fdicts_file)

    # 訓練データセットを分割し，一方を検証用に回す
    dataset_size = len(dataset)
    valid_size = int(0.01 * dataset_size) # 全体の 1% を検証用に
    train_size = dataset_size - valid_size # 残りの 99% を学習用に
    train_dataset, valid_dataset = random_split(dataset, [train_size, valid_size])

    # データセット情報をファイルに保存
    save_datasets(MODEL_DIR, train_dataset, valid_dataset)

# 訓練データおよび検証用データをミニバッチに分けて使用するための「データローダ」を用意
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

##### 学習処理の実行

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from mylib.visualizers import LossVisualizer
from mylib.utility import save_checkpoint, load_checkpoint


# 前回の試行の続きを行いたい場合は True にする -> 再開モードになる
RESTART_MODE = False


# エポック番号
INIT_EPOCH = 0 # 初期値
LAST_EPOCH = INIT_EPOCH + N_EPOCHS # 最終値

# ニューラルネットワークの作成
model = FakeFaceDetector(C=C, H=H, W=W).to(DEVICE)

# 最適化アルゴリズムの指定（ここでは SGD でなく Adam を使用）
optimizer = optim.Adam(model.parameters())

# 再開モードの場合は，前回チェックポイントから情報をロードして学習再開
if RESTART_MODE:
    INIT_EPOCH, LAST_EPOCH, model, optimizer = load_checkpoint(CHECKPOINT_EPOCH, CHECKPOINT_MODEL, CHECKPOINT_OPT, N_EPOCHS, model, optimizer)
    print('')

# 損失関数
loss_func =  nn.CrossEntropyLoss()

# 損失関数値を記録する準備
loss_viz = LossVisualizer(['train loss', 'valid loss', 'accuracy'], init_epoch=INIT_EPOCH)

# 勾配降下法による繰り返し学習
for epoch in range(INIT_EPOCH, LAST_EPOCH):

    print('Epoch {0}:'.format(epoch + 1))

    # 学習
    model.train()
    sum_loss = 0
    for X, Y in tqdm(train_dataloader):
        for param in model.parameters():
            param.grad = None
        X = X.to(DEVICE) # 入力画像
        Y = Y.to(DEVICE) # RealかFakeかの2値ラベル
        Y_pred = model(X) # 入力画像 X をニューラルネットワークに入力し，RealかFakeかの判定値 Y_pred を得る
        loss = loss_func(Y_pred, Y) # 損失関数の現在値を計算
        loss.backward() # 誤差逆伝播法により，個々のパラメータに関する損失関数の勾配（偏微分）を計算
        optimizer.step() # 勾配に沿ってパラメータの値を更新
        sum_loss += float(loss) * len(X)
    avg_loss = sum_loss / train_size
    loss_viz.add_value('train loss', avg_loss) # 訓練データに対する損失関数の値を記録
    print('train loss = {0:.6f}'.format(avg_loss))

    # 検証
    model.eval()
    sum_loss = 0
    n_failed = 0
    with torch.inference_mode():
        for X, Y in tqdm(valid_dataloader):
            X = X.to(DEVICE) # 入力画像
            Y = Y.to(DEVICE) # RealかFakeかの2値ラベル
            Y_pred = model(X)
            loss = loss_func(Y_pred, Y)
            sum_loss += float(loss) * len(X)
            n_failed += float(torch.count_nonzero(torch.argmax(Y_pred, dim=1) - Y)) # 判定結果と正解が一致していないデータの個数を数える
    avg_loss = sum_loss / valid_size
    accuracy = (valid_size - n_failed) / valid_size
    loss_viz.add_value('valid loss', avg_loss) # 検証用データに対する損失関数の値を記録
    loss_viz.add_value('accuracy', accuracy) # 検証用データに対する判定精度の値を記録
    print('valid loss = {0:.6f}'.format(avg_loss))
    print('accuracy = {0:.2f}%'.format(100 * accuracy))
    print('')

    # 現在の学習状態を一時ファイルに保存
    save_checkpoint(CHECKPOINT_EPOCH, CHECKPOINT_MODEL, CHECKPOINT_OPT, epoch+1, model, optimizer)

# 学習結果のニューラルネットワークモデルをファイルに保存
model = model.to('cpu')
torch.save(model.state_dict(), MODEL_FILE)

# 損失関数の記録をファイルに保存
loss_viz.save(v_file=os.path.join(MODEL_DIR, 'loss_graph.png'), h_file=os.path.join(MODEL_DIR, 'loss_history.csv'))

##### 学習済みニューラルネットワークモデルのロード

In [ ]:
import torch


# ニューラルネットワークモデルとその学習済みパラメータをファイルからロード
model = FakeFaceDetector(C=C, H=H, W=W)
model.load_state_dict(torch.load(MODEL_FILE))

##### テストデータセットの読み込み

In [12]:
from torch.utils.data import DataLoader
from mylib.data_io import CSVBasedDataset


# CSVファイルを読み込み, テストデータセットを用意
with open(os.path.join(MODEL_DIR, 'fdicts.pkl'), 'rb') as fdicts_file:
    fdicts = pickle.load(fdicts_file)
test_dataset = CSVBasedDataset(
    filename = TEST_DATASET_CSV,
    items = [
        'File Path', # X
        'Label', # Y
    ],
    dtypes = [
        'image', # Xの型
        'label', # Yの型
    ],
    dirname = DATA_DIR,
    fdicts = fdicts,
)
test_size = len(test_dataset)

# テストデータをミニバッチに分けて使用するための「データローダ」を用意
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

##### テスト処理

In [ ]:
import torch


model = model.to(DEVICE)
model.eval()

# テストデータセットを用いて判定精度を評価
n_failed = 0
with torch.inference_mode():
    for X, Y in tqdm(test_dataloader):
        X = X.to(DEVICE)
        Y = Y.to(DEVICE)
        Y_pred = model(X)
        n_failed += torch.count_nonzero(torch.argmax(Y_pred, dim=1) - Y) # 推定値と正解値が一致していないデータの個数を数える
    accuracy = (test_size - n_failed) / test_size
    print('accuracy = {0:.2f}%'.format(100 * accuracy))
    print('')